# WebRTC AI Voice Chat with LangChain

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/colinmcnamara/austin_langchain/blob/main/labs/LangChain_107/webrtc-ai-voice-chat/webrtc-ai-voice-chat.ipynb)


This lab will demonstrate how you can use technologies like LangChain, WebRTC, Speech to Text, and Text to Audio, to build and talk with your AI applications.

This lab relies on the following technologies:
* [LangChain](https://www.langchain.com/) - to develop AI applications
* [OpenAI Whisper](https://openai.com/index/whisper/) - to transcribe your speech
* [Suno Bark](https://github.com/suno-ai/bark) - to synthesize audio from text
* [WebRTC](https://webrtc.org/) - to allow communication between your browser application and the server running the AI application.

WebRTC is a peer to peer technology that allows you to transmit and receive audio, video, and other type of data between devices.

The LangChain AI application, OpenAI Whisper, and Suno Bark models all run on the server.

This lab requires `python` version `3.12`, however the version colab provides is `3.10`. In order to run this lab, you will need to upgrade our `python` version.

## Upgrading Python

In [ ]:
%%bash
python --version

Python 3.10.12


In [ ]:
%%capture
%%bash
apt-get install python3.12 python3-pip python3.12-distutils python3.12-dev python3.12-venv
sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.12 2

In [ ]:
%%bash
python --version

Python 3.12.3


In [ ]:
%%capture
%%bash
python -m ensurepip --upgrade
python -m pip install --upgrade setuptools


## Installing Ollama

In [ ]:
%%bash
curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
%%bash --bg
ollama serve &> ollama.log 2>&1


Ollama takes a minute to start and get running so we wait...

In [ ]:
!until ollama list; do echo waiting for ollama; sleep 5; done

NAME	ID	SIZE	MODIFIED 


We will now download Phi3 small language model.

In [ ]:
%%bash --bg
ollama pull phi3 >> phi3.log 2>&1

 This step also takes some time, so we wait some more...

In [ ]:
!until ollama list | grep phi3; do echo waiting for phi; sleep 5; done

waiting for phi
waiting for phi
waiting for phi
waiting for phi
waiting for phi
waiting for phi
phi3:latest	a2c89ceaed85	2.3 GB	3 seconds ago	


Next step, download the source code for the WebRTC AI Voice Chat demo and prepare our workspace

In [ ]:
%%bash
git clone https://github.com/lalanikarim/webrtc-ai-voice-chat.git

Cloning into 'webrtc-ai-voice-chat'...


In [ ]:
%cd webrtc-ai-voice-chat

/content/webrtc-ai-voice-chat


Prepare our Python workspace and install project's dependencies. This will take some time. You can check pip.log file to monitor the progress....

In [ ]:
%%bash
python -m venv .venv
source .venv/bin/activate
pip install -r requirements.txt > ../pip.log

We now check to make sure Ollama is still running

In [ ]:
%%bash
ollama list

NAME       	ID          	SIZE  	MODIFIED      
phi3:latest	a2c89ceaed85	2.3 GB	8 minutes ago	


Now we start the Python server. This will also start the download of the Whisper and Bark models.

In [ ]:
%%bash --bg
source .venv/bin/activate
python -m server --port 8888 > ../webrtc.log 2>&1

Since the server takes a few seconds to star up, let's wait some more...

In [ ]:
!until curl -I http://localhost:8888; do echo waiting for server; sleep 5; done

curl: (7) Failed to connect to localhost port 8888 after 0 ms: Connection refused
waiting for server
curl: (7) Failed to connect to localhost port 8888 after 0 ms: Connection refused
waiting for server
curl: (7) Failed to connect to localhost port 8888 after 0 ms: Connection refused
waiting for server
curl: (7) Failed to connect to localhost port 8888 after 0 ms: Connection refused
waiting for server
curl: (7) Failed to connect to localhost port 8888 after 0 ms: Connection refused
waiting for server
curl: (7) Failed to connect to localhost port 8888 after 0 ms: Connection refused
waiting for server
curl: (7) Failed to connect to localhost port 8888 after 0 ms: Connection refused
waiting for server
HTTP/1.1 200 OK
Content-Type: text/html; charset=utf-8
Content-Length: 4208
Date: Wed, 08 May 2024 22:41:09 GMT
Server: Python/3.12 aiohttp/3.9.5



Get the public IP for your Colab instance. We will need this IP to gain access to our app in the browser later.

In [ ]:
%%bash
curl ipv4.icanhazip.com 2>/dev/null

34.171.216.23


Start the local tunnel

In [ ]:
%%bash --bg
npx localtunnel --port 8888 > ../localtunnel.log

Grab the url generated by the local tunnel

In [ ]:
%%bash
cat ../localtunnel.log

your url is: https://tiny-buttons-carry.loca.lt


Note: Chrome works best for this demo. Firefox may not recover from WebRTC failures reliably.

Note: At anytime, if Ollama or the Python server stop running, you'll need to restart them by running the appropriate cells. This might happen for a couple of times before the server runs with more stability.